# Diverse Model Ensemble: [32,16] MLP + LightGBM + [64,32] MLP

**Rationale**: Single-model optimization has plateaued. CV-LB correlation has broken down:
- exp_007 ([32,16]): CV 0.0093, LB 0.0932 (BEST LB)
- exp_009 ([16]): CV 0.0092, LB 0.0936 (WORSE LB despite better CV)

**Hypothesis**: Ensembling diverse models may improve generalization by:
1. Reducing variance through averaging
2. Capturing different patterns with different model types
3. Providing more robust predictions on unseen solvents

**Ensemble composition**:
- [32,16] MLP (best LB model)
- LightGBM (different model family, captures non-linearities differently)
- [64,32] MLP (slightly more complex, may capture additional patterns)

**Weighting**: Start with equal weights, can tune based on CV performance.

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import lightgbm as lgb
import tqdm
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.set_default_dtype(torch.double)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [2]:
# Load data and features
DATA_PATH = '/home/data'

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT NAME"]]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

# Load feature lookups
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
DRFP_DF = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)
drfp_variance = DRFP_DF.var()
nonzero_variance_cols = drfp_variance[drfp_variance > 0].index.tolist()
DRFP_FILTERED = DRFP_DF[nonzero_variance_cols]

print(f'Spange: {SPANGE_DF.shape}, DRFP filtered: {DRFP_FILTERED.shape}')

Spange: (26, 13), DRFP filtered: (24, 122)


In [3]:
# Combined Featurizer for both MLP and LightGBM
class CombinedFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.drfp_df = DRFP_FILTERED
        self.feats_dim = 2 + 3 + self.spange_df.shape[1] + self.drfp_df.shape[1]

    def featurize(self, X, flip=False):
        X_vals = X[["Residence Time", "Temperature"]].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            A_drfp = self.drfp_df.loc[X["SOLVENT A NAME"]].values
            B_drfp = self.drfp_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
                X_drfp = B_drfp * (1 - (1-pct)) + A_drfp * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
                X_drfp = A_drfp * (1 - pct) + B_drfp * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
            X_drfp = self.drfp_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange, X_drfp])
    
    def featurize_torch(self, X, flip=False):
        return torch.tensor(self.featurize(X, flip))

print(f'Feature dimension: {CombinedFeaturizer().feats_dim}')

Feature dimension: 140


In [4]:
# MLP Model with configurable architecture
class MLPModel(nn.Module):
    def __init__(self, input_dim, hidden_dims=[32, 16], output_dim=3, dropout=0.05):
        super(MLPModel, self).__init__()
        layers = [nn.BatchNorm1d(input_dim)]
        prev_dim = input_dim
        for h_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, h_dim),
                nn.BatchNorm1d(h_dim),
                nn.ReLU(),
                nn.Dropout(dropout)
            ])
            prev_dim = h_dim
        layers.append(nn.Linear(prev_dim, output_dim))
        layers.append(nn.Sigmoid())
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

# MLP Ensemble
class MLPEnsemble:
    def __init__(self, hidden_dims=[32, 16], n_models=5, data='single'):
        self.hidden_dims = hidden_dims
        self.n_models = n_models
        self.data_type = data
        self.featurizer = CombinedFeaturizer(mixed=(data=='full'))
        self.models = []

    def train_model(self, X_train, y_train, epochs=200, batch_size=32, lr=5e-4):
        X_std = self.featurizer.featurize_torch(X_train, flip=False)
        y_vals = torch.tensor(y_train.values)
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize_torch(X_train, flip=True)
            X_all = torch.cat([X_std, X_flip], dim=0)
            y_all = torch.cat([y_vals, y_vals], dim=0)
        else:
            X_all = X_std
            y_all = y_vals
            
        input_dim = X_all.shape[1]
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        for i in range(self.n_models):
            torch.manual_seed(42 + i * 13)
            np.random.seed(42 + i * 13)
            
            model = MLPModel(input_dim, hidden_dims=self.hidden_dims, dropout=0.05).to(device)
            model.train()
            self.models.append(model)
            
            dataset = TensorDataset(X_all, y_all)
            loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
            
            optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
            criterion = nn.HuberLoss()
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20)
            
            for epoch in range(epochs):
                epoch_loss = 0.0
                for inputs, targets in loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, targets)
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                    optimizer.step()
                    epoch_loss += loss.item() * inputs.size(0)
                scheduler.step(epoch_loss / len(dataset))

    def predict(self, X):
        device = next(self.models[0].parameters()).device
        
        if self.data_type == 'full':
            X_std = self.featurizer.featurize_torch(X, flip=False).to(device)
            X_flip = self.featurizer.featurize_torch(X, flip=True).to(device)
            pred_sum = torch.zeros((len(X), 3)).to(device)
            with torch.no_grad():
                for model in self.models:
                    model.eval()
                    pred_sum += (model(X_std) + model(X_flip)) * 0.5
            avg_pred = pred_sum / self.n_models
        else:
            X_std = self.featurizer.featurize_torch(X).to(device)
            pred_sum = torch.zeros((len(X), 3)).to(device)
            with torch.no_grad():
                for model in self.models:
                    model.eval()
                    pred_sum += model(X_std)
            avg_pred = pred_sum / self.n_models
        return avg_pred.cpu().numpy()

print('MLPEnsemble defined')

MLPEnsemble defined


In [5]:
# LightGBM Model
class LGBMModel:
    def __init__(self, data='single'):
        self.data_type = data
        self.featurizer = CombinedFeaturizer(mixed=(data=='full'))
        self.models = []  # One model per target
        self.params = {
            'objective': 'regression',
            'metric': 'mse',
            'boosting_type': 'gbdt',
            'num_leaves': 31,
            'learning_rate': 0.05,
            'feature_fraction': 0.8,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'verbose': -1,
            'seed': 42
        }

    def train_model(self, X_train, y_train, num_boost_round=200):
        X_feat = self.featurizer.featurize(X_train)
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize(X_train, flip=True)
            X_all = np.vstack([X_feat, X_flip])
            y_all = np.vstack([y_train.values, y_train.values])
        else:
            X_all = X_feat
            y_all = y_train.values
        
        self.models = []
        for i in range(3):  # 3 targets
            train_data = lgb.Dataset(X_all, label=y_all[:, i])
            model = lgb.train(self.params, train_data, num_boost_round=num_boost_round)
            self.models.append(model)

    def predict(self, X):
        if self.data_type == 'full':
            X_std = self.featurizer.featurize(X, flip=False)
            X_flip = self.featurizer.featurize(X, flip=True)
            preds = np.zeros((len(X), 3))
            for i, model in enumerate(self.models):
                preds[:, i] = (model.predict(X_std) + model.predict(X_flip)) / 2
        else:
            X_feat = self.featurizer.featurize(X)
            preds = np.zeros((len(X), 3))
            for i, model in enumerate(self.models):
                preds[:, i] = model.predict(X_feat)
        return np.clip(preds, 0, 1)

print('LGBMModel defined')

LGBMModel defined


In [6]:
# Diverse Ensemble combining MLP [32,16] + LightGBM + MLP [64,32]
class DiverseEnsemble:
    def __init__(self, data='single', weights=[0.5, 0.25, 0.25]):
        self.data_type = data
        self.weights = weights  # [MLP_32_16, LGBM, MLP_64_32]
        self.mlp_32_16 = None
        self.lgbm = None
        self.mlp_64_32 = None

    def train_model(self, X_train, y_train):
        # Train MLP [32,16] - best LB model
        self.mlp_32_16 = MLPEnsemble(hidden_dims=[32, 16], n_models=5, data=self.data_type)
        self.mlp_32_16.train_model(X_train, y_train, epochs=200)
        
        # Train LightGBM - different model family
        self.lgbm = LGBMModel(data=self.data_type)
        self.lgbm.train_model(X_train, y_train, num_boost_round=200)
        
        # Train MLP [64,32] - slightly more complex
        self.mlp_64_32 = MLPEnsemble(hidden_dims=[64, 32], n_models=5, data=self.data_type)
        self.mlp_64_32.train_model(X_train, y_train, epochs=200)

    def predict(self, X):
        pred_mlp_32_16 = self.mlp_32_16.predict(X)
        pred_lgbm = self.lgbm.predict(X)
        pred_mlp_64_32 = self.mlp_64_32.predict(X)
        
        # Weighted average
        ensemble_pred = (
            self.weights[0] * pred_mlp_32_16 +
            self.weights[1] * pred_lgbm +
            self.weights[2] * pred_mlp_64_32
        )
        return np.clip(ensemble_pred, 0, 1)

print('DiverseEnsemble defined: MLP[32,16] + LightGBM + MLP[64,32]')
print(f'Weights: MLP[32,16]=0.5, LGBM=0.25, MLP[64,32]=0.25')

DiverseEnsemble defined: MLP[32,16] + LightGBM + MLP[64,32]
Weights: MLP[32,16]=0.5, LGBM=0.25, MLP[64,32]=0.25


In [7]:
# Run CV for single solvent
print("\n--- TASK 0: Single Solvent (24 folds) ---")
X, Y = load_data("single_solvent")
split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []
all_actuals = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=24):
    (train_X, train_Y), (test_X, test_Y) = split
    model = DiverseEnsemble(data='single', weights=[0.5, 0.25, 0.25])
    model.train_model(train_X, train_Y)
    predictions = model.predict(test_X)
    for row_idx, row in enumerate(predictions):
        all_predictions.append({"task": 0, "fold": fold_idx, "row": row_idx,
            "target_1": row[0], "target_2": row[1], "target_3": row[2]})
    all_actuals.append(test_Y.values)

submission_single = pd.DataFrame(all_predictions)
actuals_single = np.vstack(all_actuals)
preds_single = submission_single[['target_1', 'target_2', 'target_3']].values
mse_single = np.mean((actuals_single - preds_single) ** 2)
print(f'\nSingle Solvent MSE: {mse_single:.6f}')


--- TASK 0: Single Solvent (24 folds) ---


  0%|          | 0/24 [00:00<?, ?it/s]

  4%|▍         | 1/24 [02:06<48:30, 126.53s/it]

  8%|▊         | 2/24 [04:08<45:19, 123.62s/it]

 12%|█▎        | 3/24 [06:10<43:08, 123.28s/it]

...

 92%|█████████▏| 22/24 [49:40<04:13, 126.95s/it]

 96%|█████████▌| 23/24 [51:49<02:07, 127.34s/it]

100%|██████████| 24/24 [53:50<00:00, 125.47s/it]

100%|██████████| 24/24 [53:50<00:00, 134.60s/it]


Single Solvent MSE: 0.009469


In [8]:
# Run CV for full data
print("\n--- TASK 1: Full Data (13 folds) ---")
X, Y = load_data("full")
split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions_full = []
all_actuals_full = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=13):
    (train_X, train_Y), (test_X, test_Y) = split
    model = DiverseEnsemble(data='full', weights=[0.5, 0.25, 0.25])
    model.train_model(train_X, train_Y)
    predictions = model.predict(test_X)
    for row_idx, row in enumerate(predictions):
        all_predictions_full.append({"task": 1, "fold": fold_idx, "row": row_idx,
            "target_1": row[0], "target_2": row[1], "target_3": row[2]})
    all_actuals_full.append(test_Y.values)

submission_full = pd.DataFrame(all_predictions_full)
actuals_full = np.vstack(all_actuals_full)
preds_full = submission_full[['target_1', 'target_2', 'target_3']].values
mse_full = np.mean((actuals_full - preds_full) ** 2)
print(f'\nFull Data MSE: {mse_full:.6f}')


--- TASK 1: Full Data (13 folds) ---


  0%|          | 0/13 [00:00<?, ?it/s]

  8%|▊         | 1/13 [07:08<1:25:47, 428.97s/it]

 15%|█▌        | 2/13 [14:11<1:17:56, 425.17s/it]

 23%|██▎       | 3/13 [21:23<1:11:24, 428.41s/it]

...

 85%|████████▍ | 11/13 [1:14:21<13:23, 401.81s/it]

 92%|█████████▏| 12/13 [1:21:30<06:49, 409.88s/it]

100%|██████████| 13/13 [1:30:21<00:00, 446.54s/it]

100%|██████████| 13/13 [1:30:21<00:00, 417.01s/it]


Full Data MSE: 0.008487


In [9]:
# Calculate overall MSE
n_single = len(actuals_single)
n_full = len(actuals_full)
overall_mse = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)

print(f'\n=== FINAL RESULTS ===')
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single})')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')
print(f'Overall MSE: {overall_mse:.6f}')
print(f'\n=== COMPARISON ===')
print(f'  exp_007 [32,16] alone: CV 0.009262, LB 0.0932 (BEST LB)')
print(f'  exp_009 [16] alone: CV 0.009192, LB 0.0936')
print(f'  This ensemble: CV {overall_mse:.6f}')
print(f'\nExpected LB (using 10x ratio): {overall_mse * 10:.4f}')


=== FINAL RESULTS ===
Single Solvent MSE: 0.009469 (n=656)
Full Data MSE: 0.008487 (n=1227)
Overall MSE: 0.008829

=== COMPARISON ===
  exp_007 [32,16] alone: CV 0.009262, LB 0.0932 (BEST LB)
  exp_009 [16] alone: CV 0.009192, LB 0.0936
  This ensemble: CV 0.008829

Expected LB (using 10x ratio): 0.0883


In [10]:
# Save submission
submission = pd.concat([submission_single, submission_full])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)
print(f'Submission saved: {submission.shape}')
print(submission.head())

Submission saved: (1883, 7)
    index  task  fold  row  target_1  target_2  target_3
id                                                      
0       0     0     0    0  0.012764  0.012860  0.914583
1       1     0     0    1  0.025176  0.022548  0.874224
2       2     0     0    2  0.036707  0.031185  0.843390
3       3     0     0    3  0.053521  0.044479  0.806742
4       4     0     0    4  0.065304  0.059500  0.751237
